In [1]:
# Importing the required libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

import pickle

import warnings
warnings.filterwarnings('ignore')

**Metric:** Function that compares the predictions with the actual values and outputs a single number that tells how good the predictions are.

### 1. Prerequisite

#### Data ingestion and preparation

In [2]:
df = pd.read_csv('data.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

#### Setting validation framework

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

#### Numerical and categorical features

In [4]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

#### Model training and validation

In [5]:
# Function to train the model
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [6]:
# Function to predict
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [7]:
# Parameters
C = 1.0
n_splits = 5

In [8]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.009


In [9]:
scores

[0.8435760768029139,
 0.8436775758177725,
 0.8311780052177403,
 0.8301724275756219,
 0.8526952401870038]

In [10]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

### 2. Save and Load Model

#### Save Model

In [11]:
# Constructing the model file name
output_file_name = f'model_C={C}.bin'
output_file_name

'model_C=1.0.bin'

In [12]:
# Opening file and saving the model
with open(output_file_name , 'wb') as f_out:
    pickle.dump((dv, model), f_out)

#### Load Model

In [13]:
model_name = 'model_C=1.0.bin'

In [14]:
# Opening file and loading the model
with open(model_name, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [15]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

#### Testing Loaded Model

In [16]:
# Selecting one random customer
customer = df[categorical+numerical].iloc[100].to_dict()
customer

{'gender': 'male',
 'seniorcitizen': 0,
 'partner': 'no',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'no',
 'onlinesecurity': 'no_internet_service',
 'onlinebackup': 'no_internet_service',
 'deviceprotection': 'no_internet_service',
 'techsupport': 'no_internet_service',
 'streamingtv': 'no_internet_service',
 'streamingmovies': 'no_internet_service',
 'contract': 'month-to-month',
 'paperlessbilling': 'no',
 'paymentmethod': 'mailed_check',
 'tenure': 1,
 'monthlycharges': 20.2,
 'totalcharges': 20.2}

#### AUC Interpretation
Probabilty that randomly selected churned people[postive] is higher that not churned perople[negative].

In [17]:
neg = y_pred[y_val == 0]
pos = y_pred[y_val == 1]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1409 but corresponding boolean dimension is 1126

In [ ]:
import random

In [ ]:
n = 100000
success = 0

for i in range(n):
    
    pos_idx = random.randint(0, len(pos) - 1)
    neg_idx = random.randint(0, len(neg) - 1)
    
    if pos[pos_idx] > neg[neg_idx]:
        success += 1
        
success/n

- **Metric:** a single number that describe the preformance of model.
- **Accuracy:** fractions of correct answer; sometime it's misleading.
- **Precision & Recall:** less misleading when class is imbalanced.
- **ROC Curve:** a way to evaluate the performance at all thresholds; okay to use with imbalance
- **K-Fold CV:** more reliable estimates the performance(mean + std)  

- Check the precision and recall of the dummy classifier that always predict 'FALSE'
- F1 score = 2 * P * R / (P + R)
- Evaluate precision and recall at different thresholds, plot P vs R - this way you will get precision and recall curve (similar to ROC curve)
- Are under the PR curve is also useful metric

In [ ]:
customer_X = dv.transform([customer])
customer_X

In [ ]:
model.predict_proba(customer_X)[0, 1]